In [ ]:
from typing import List, Any

import numpy as np

## Hitrate@K

Такая метрика должна вернуть 1, только если множество релевантных и top-K рекомендаций имеет пересечение

In [ ]:
def user_intersection(y_rel: List[Any], y_rec: List[Any], k: int = 10) -> int:
    """
    :param y_rel: relevant items
    :param y_rec: recommended items
    :param k: number of top recommended items
    :return: number of items in intersection of y_rel and y_rec (truncated to top-K)
    """
    return len(set(y_rec[:k]).intersection(set(y_rel)))


def user_hitrate(y_rel: List[Any], y_rec: List[Any], k: int = 10) -> int:
    """
    :param y_rel: relevant items
    :param y_rec: recommended items
    :param k: number of top recommended items
    :return: 1 if top-k recommendations contains at lease one relevant item
    """
    return int(user_intersection(y_rel, y_rec, k) > 0)

## Precision@K

Метрика является отношением числа релевантных рекомендаций к числу рекомендованных объектов

Precision@K = $\frac{|y_{rel} \cap y_{rec}|}{|y_{rec}|} = \frac{|y_{rel} \cap y_{rec}|}{K}$

In [ ]:
def user_precision(y_rel: List[Any], y_rec: List[Any], k: int = 10) -> float:
    """
    :param y_rel: relevant items
    :param y_rec: recommended items
    :param k: number of top recommended items
    :return: percentage of relevant items through recommendations
    """
    return user_intersection(y_rel, y_rec, k) / k

## Recall@K

Метрика является отношением числа релевантных рекомендаций к числу релевантных объектов

Recall@K = $\frac{|y_{rel} \cap y_{rec}|}{|y_{rel}|}$

In [ ]:
def user_recall(y_rel: List[Any], y_rec: List[Any], k: int = 10) -> float:
    """
    :param y_rel: relevant items
    :param y_rec: recommended items
    :param k: number of top recommended items
    :return: percentage of found relevant items through recommendations
    """
    return user_intersection(y_rel, y_rec, k) / len(set(y_rel))

## AP@K

Для подсчета AP@K нужно просуммировать Precision@k по индексам k от 1 до K только для **релевантных** элементов, деленому на K

Например, если из топ-3 рекомендаций только первый релевантный, то AP@3 = $\frac{1}{3} (1 / 1 + 0 + 0) = \frac{1}{3}$

Если релевантный только последний, то AP@3 = $\frac{1}{3} (0 + 0 + 1 / 3) = \frac{1}{9}$

In [ ]:
def user_ap(y_rel: List[Any], y_rec: List[Any], k: int = 10) -> float:
    """
    :param y_rel: relevant items
    :param y_rec: recommended items
    :param k: number of top recommended items
    :return: average precision metric for user recommendations
    """
    return np.sum([
        user_precision(y_rel, y_rec, idx + 1)
        for idx, item in enumerate(y_rec[:k]) if item in y_rel
    ]) / k


## RR@K

Метрика равна $\frac{1}{pos}$, где pos – индекс первой релевантной позиции (нумерация с 1)

Если же релевантных объектов вообще нет, то метрика равна 0

In [ ]:
def user_rr(y_rel: List[Any], y_rec: List[Any], k: int = 10) -> float:
    """
    :param y_rel: relevant items
    :param y_rec: recommended items
    :param k: number of top recommended items
    :return: reciprocal rank for user recommendations
    """
    for idx, item in enumerate(y_rec[:k]):
        if item in y_rel:
            return 1 / (idx + 1)
    return 0

## NDCG@K

Метрика DCG@K является взвешенной суммой по тем позициям, где рекомендованный объект релевантный (вес равен $\frac{1}{\log{pos}}$)

В качестве нормировки посчитаем метрику iDCG@K, которая равна взвешенной сумме всех позиций (но не больше числа релевантных объектов)

Тоогда метрика NDCG@K = $\frac{DCG@K}{iDCG@K}$

In [ ]:
def user_ndcg(y_rel: List[Any], y_rec: List[Any], k: int = 10) -> float:
    """
    :param y_rel: relevant items
    :param y_rec: recommended items
    :param k: number of top recommended items
    :return: ndcg metric for user recommendations
    """
    dcg = sum([1. / np.log2(idx + 2) for idx, item in enumerate(y_rec[:k]) if item in y_rel])
    idcg = sum([1. / np.log2(idx + 2) for idx, _ in enumerate(zip(y_rel, np.arange(k)))])
    return dcg / idcg

